# Исследование уровня потребительской лояльности  для       телекоммуникационной компании

**Цель исследования**

Определить уровень потребительской лояльности, или NPS (от англ. Net Promoter Score),  среди клиентов из России для большой  телекоммуникационной компании, оказывающей услуги на территории всего СНГ. 

**Описание данных**

Данные получены путем опроса среди клиентов, которым задавали следующий кдассический вопрос: «Оцените по шкале от 1 до 10 вероятность того, что вы порекомендуете компанию друзьям и знакомым». Оценки разделили на три группы:

- 9-10 баллов — «cторонники» (англ. promoters);
- 7-8 баллов — «нейтралы» (англ. passives);
- 0-6 баллов — «критики» (англ. detractors).

Итоговое значение NPS рассчитывается по формуле: % «сторонников» - % «критиков».
Таким образом, значение этого показателя варьируется от -100% (когда все клиенты «критики») до 100% (когда все клиенты лояльны к сервису). 

Данные по исследованию выгрузили в SQLite.

**Ход исследования**

- Подключимся к базе данных в SQLite. 

- Напишем запрос, который выгрузит следующие данные:

  - user_id - идентификатор клиента;
  - lt_day - количество дней «жизни» клиента;
  - is_new - поле хранит информацию о том, является ли клиент новым;
  - age - возраст;
  - gender_segment - пол (для удобства работы с полем преобразуем значения в текстовый вид);
  - os_name тип операционной системы;
  - cpe_type_name - тип устройства;
  - country - страна проживания;
  - city - город проживания;
  - age_segment - возрастной сегмент;
  - traffic_segment - сегмент по объёму потребляемого трафика;
  - lifetime_segment сегмент по количеству дней «жизни»;
  - nps_score - оценка клиента в NPS-опросе;
  - nps_group - поле хранит информацию о том, к какой группе относится оценка клиента в опросе.
 
- Создадим дашборд в Tableau, который представит информацию о текущем уровне NPS среди клиентов и покажет, как этот уровень меняется в зависимости от пользовательских признаков. 
 
- Ответим на следующие вопросы с помощью дашборда:

  - Как распределены участники опроса по возрасту и полу? Каких пользователей больше: новых или старых? Пользователи из каких городов активнее участвовали в опросе?
  - Какие группы пользователей наиболее лояльны к сервису? Какие менее?
  - Какой общий NPS среди всех опрошенных?
  - Как можно описать клиентов, которые относятся к группе cторонников (англ. promoters)?

Ответы на вопросы оформим в виде презентации. 

In [1]:
# Импортируем библиотеки.
import os
import pandas as pd
import numpy as np

from sqlalchemy import create_engine

In [2]:
# Подключаемся к базе данных.
path_to_db_local = 'telecomm_csi.db'
path_to_db_platform = '/Users/marina/CSV_files/telecomm_csi.db'
path_to_db = None

if os.path.exists(path_to_db_local):
    path_to_db = path_to_db_local
elif os.path.exists(path_to_db_platform):
    path_to_db = path_to_db_platform
else:
    raise Exception('Файл с базой данных SQLite не найден!')

if path_to_db:
    engine = create_engine(f'sqlite:///{path_to_db}', echo=False)

In [3]:
# Пишем запрос к базе данных.
query = """
SELECT u.user_id, 
       u.lt_day, 
        CASE 
            WHEN u.lt_day <= 365 THEN 'новый' 
            ELSE 'постоянный' 
        END AS is_new,
       u.age, 
        CASE 
            WHEN u.gender_segment = 1 THEN 'женский'
            WHEN u.gender_segment = 0 THEN 'мужской'
        END AS gender_segment,
       u.os_name, 
       u.cpe_type_name,
       l.country,
       l.city,
        SUBSTR(a.title, 3) AS age_segment,
        SUBSTR(t.title, 3) AS traffic_segment,
        SUBSTR(ls.title, 3) AS lifetime_segment,
       u.nps_score,
        CASE 
            WHEN u.nps_score >= 9 THEN 'сторонники'
            WHEN u.nps_score >= 7 THEN 'нейтралы'
            ELSE 'критики'
        END AS nps_group
FROM user AS u
LEFT JOIN location AS l ON u.location_id=l.location_id
LEFT JOIN age_segment AS a ON u.age_gr_id=a.age_gr_id
LEFT JOIN traffic_segment AS t ON u.tr_gr_id=t.tr_gr_id
LEFT JOIN lifetime_segment AS ls ON u.lt_gr_id=ls.lt_gr_id;
"""

In [4]:
# Создаем датафрейм по данным запроса.
df = pd.read_sql(query, engine)
df.head(3)

,user_id,lt_day,is_new,age,gender_segment,os_name,cpe_type_name,country,city,age_segment,traffic_segment,lifetime_segment,nps_score,nps_group
0,A001A2,2320,постоянный,45.0,женский,ANDROID,SMARTPHONE,Россия,Уфа,45-54,1-5,36+,10,сторонники
1,A001WF,2344,постоянный,53.0,мужской,ANDROID,SMARTPHONE,Россия,Киров,45-54,1-5,36+,10,сторонники
2,A003Q7,467,постоянный,57.0,мужской,ANDROID,SMARTPHONE,Россия,Москва,55-64,20-25,13-24,10,сторонники


Проверим качество данных в стобцах lt_day и nps_score.

In [5]:
df['lt_day'].isna().sum()

0

In [6]:
df['nps_score'].isna().sum()

0

In [7]:
print(df['lt_day'].unique())
len(df['lt_day'].unique())

[2320 2344  467 ... 8202 7138 6861]


6950

In [8]:
print(df['nps_score'].unique())
len(df['nps_score'].unique())

[10  6  9  2  5  8  1  4  7  3]


10

In [9]:
df['lt_day'].sort_values().head(20)

100152   -21
325212   -13
407160   -13
228087   -13
455359   -12
385897   -12
401226   -11
7666      -8
492382    -7
347784    -6
71577     -4
53997     -2
425927    -1
284966     0
480641     0
403933     0
103223     0
127634     1
430205     1
314217     1
Name: lt_day, dtype: int64

Cтолбец lt_day имеет ряд отрицательных значений, вероятно, это техническая ошибка, учту это в дальнейшей работе.

In [10]:
# Сохраняем датафрейм в csv-файл для дальнейшей работы.
df.to_csv('telecomm_csi_tableau.csv', index=False)

Презентация:


https://public.tableau.com/app/profile/marina.orlenko/viz/TableauMulti-partProject2/sheet26?publish=yes